In [14]:
import pandas as pd
import numpy as np
import ccxt
from typing import Dict, Any, List, Tuple
from popquants_grid_allocator import build_grid

# ==============================
# ฟังก์ชันช่วย: ดึง min_notional / minQty จาก Binance ผ่าน ccxt
# ==============================
def fetch_ccxt_constraints(symbol: str, exchange_id: str = "binance") -> Dict[str, float]:
    """
    คืนค่า dict: {
        'min_notional': float หรือ None,
        'min_qty': float หรือ None
    }
    """
    ex = getattr(ccxt, exchange_id)({"enableRateLimit": True})
    ex.load_markets()
    m = ex.market(symbol)

    min_notional = None
    min_qty = None

    # 1) พยายามใช้ unified limits ก่อน (ถ้ามี)
    limits = m.get("limits", {})
    cost = limits.get("cost", {}) if isinstance(limits, dict) else {}
    if isinstance(cost, dict) and cost.get("min") is not None:
        try:
            min_notional = float(cost["min"])
        except Exception:
            pass

    amount = limits.get("amount", {}) if isinstance(limits, dict) else {}
    if isinstance(amount, dict) and amount.get("min") is not None:
        try:
            min_qty = float(amount["min"])
        except Exception:
            pass

    # 2) ถ้ายังไม่มี ให้ดูจาก info.filters ของ Binance
    if min_notional is None or min_qty is None:
        filters = m.get("info", {}).get("filters", []) or []
        for f in filters:
            ftype = f.get("filterType")
            if ftype in ("MIN_NOTIONAL", "NOTIONAL") and min_notional is None:
                raw = f.get("minNotional") or f.get("notional")
                if raw is not None:
                    try:
                        min_notional = float(raw)
                    except Exception:
                        pass
            if ftype == "LOT_SIZE" and min_qty is None:
                raw = f.get("minQty")
                if raw is not None:
                    try:
                        min_qty = float(raw)
                    except Exception:
                        pass

    return {
        "min_notional": float(min_notional) if min_notional is not None else None,
        "min_qty": float(min_qty) if min_qty is not None else None,
    }


# ==============================
# โหลดไฟล์ CSV ที่ได้จากสคริปต์ Monte Carlo
# ==============================
df = pd.read_csv("macro_montecarlo.csv")

# ==============================
# 1) ดึงแถวสรุป (summary row) — แถวสุดท้าย
# ==============================
summary = df.iloc[-1]   # แถวสุดท้ายเป็น summary row
spot = float(summary["spot"])
band_low = float(summary["band_low"])
band_high = float(summary["band_high"])

# ==============================
# 2) ดึงตัวอย่างราคาต่ำสุดต่อเส้นทาง (ใช้สำหรับ equal-prob)
#    เลือกเฉพาะแถวที่ path_id ไม่เป็น NaN
# ==============================
path_df = df[df["path_id"].notna()]
mc_mins_samples = path_df["min_price"].to_numpy(dtype=float)

# ==============================
# 3) คำนวณ w_min จาก ccxt (Binance) ให้เหมาะกับ SYMBOL
#    - ใช้ min_notional (เช่น ~10 USDT) เป็นฐาน
#    - กันกรณีเล็กกว่าขั้นต่ำเชิงปริมาณ: max(min_notional, spot * minQty)
#    - ตั้ง fallback ถ้าหาไม่ได้
# ==============================
SYMBOL = "XRP/USDT"
ccxt_info = fetch_ccxt_constraints(SYMBOL, exchange_id="binance")
min_notional = ccxt_info.get("min_notional")
min_qty = ccxt_info.get("min_qty")

# ฐานขั้นต่ำตามกติกา
w_min_ccxt = None
candidates = []
if min_notional is not None:
    candidates.append(min_notional)
if (min_qty is not None) and spot > 0:
    candidates.append(spot * min_qty)

if candidates:
    w_min_ccxt = float(max(candidates))
else:
    # Fallback ถ้าดึงไม่ได้
    w_min_ccxt = 10.0

# เพื่อความปลอดภัย: เผื่อ buffer เล็กน้อย 1% (กันกรณีราคา/ทศนิยมปัดแล้วต่ำกว่า)
w_min_ccxt = float(np.ceil(w_min_ccxt * 1.01 * 100) / 100.0)

# ตั้ง w_max ให้ไม่ต่ำกว่า w_min
w_max_cfg = 50.0
if w_max_cfg < w_min_ccxt:
    w_max_cfg = w_min_ccxt

print("[i] ข้อจำกัดจากตลาด (ccxt/Binance):")
print(f"    min_notional ≈ {min_notional if min_notional is not None else 'N/A'} USDT")
print(f"    min_qty      ≈ {min_qty if min_qty is not None else 'N/A'} {SYMBOL.split('/')[0]}")
print(f"    spot         = {spot:.6f} → spot*minQty ≈ {spot*min_qty:.4f} USDT" if min_qty is not None else f"    spot         = {spot:.6f}")
print(f"    → เลือก w_min = {w_min_ccxt:.2f} USDT  (w_max = {w_max_cfg:.2f})")

# ==============================
# 4) เรียก build_grid เพื่อสร้างแผนกริด (ใช้ w_min จาก ccxt)
# ==============================
budget = 100.0

result = build_grid(
    spot=spot,
    band_low=band_low,
    band_high=band_high,
    budget_usd=budget,
    K=10,                        # จำนวนชั้นกริด (กำหนดคงที่เป็น 20 ในตัวอย่างนี้)
    method="equal_prob",         # กระจายจุดกริดตาม quantile จากตัวอย่าง min ของ MC
    mc_mins_samples=mc_mins_samples,
    alpha=0.7,                   # กันสำรองงบไว้ 30% (ใช้ทุนคาดหวังไม่เกิน 70% ของงบ)
    w_min=w_min_ccxt,            # << ใช้ค่าขั้นต่ำต่อคำสั่งจาก ccxt
    w_max=w_max_cfg,             # << ให้ไม่ต่ำกว่า w_min
    fee_rate=0.0004,             # ค่าธรรมเนียมต่อรอบ (ประมาณ 0.04%)
    tp_pct=0.01,                 # เป้า TP ต่อคำสั่ง 1% จากราคาเข้าซื้อ
    zone_near=0.05,              # โซนใกล้ (±5% รอบ spot)
    zone_mid=0.15,               # โซนกลาง (±15% รอบ spot)
    weight_scheme="far_heavier"  # เน้นน้ำหนักชั้นที่ไกลจาก spot มากกว่า (FAR หนักสุด)
)

# ==============================
# 5) แสดงผลลัพธ์
# ==============================
print("\n=== แผนวางกริด (อ่านจาก macro_montecarlo.csv) ===")
print(f"ราคาปัจจุบัน (Spot): {result['spot']:.6f} | ช่วงคาดการณ์ (Band): {result['band_low']:.6f} → {result['band_high']:.6f}")
print("จำนวนชั้นในแต่ละโซน (Zone counts):", result["zone_counts"])
print("สรุปภาพรวม (Totals):", result["totals"])

print("\nคำสั่งซื้อ (Orders):")
print("ZONE | buy_price | usd_alloc | coin_size | tp_price | net_pct_est")
for r in result["orders"]:
    print(f"{r['zone']:>4} | {r['buy_price']:.6f} | {r['usd_alloc']:>8.2f} "
          f"| {r['coin_size']:>9.6f} | {r['tp_price']:.6f} | {r['net_pct_est']:.4%}")

# บันทึกคำสั่งลงไฟล์ CSV
pd.DataFrame(result["orders"]).to_csv("grid_plan.csv", index=False)
print("\n[i] บันทึกแผนกริดลงไฟล์: grid_plan.csv เรียบร้อยแล้ว")


[i] ข้อจำกัดจากตลาด (ccxt/Binance):
    min_notional ≈ 5.0 USDT
    min_qty      ≈ 0.1 XRP
    spot         = 3.112700 → spot*minQty ≈ 0.3113 USDT
    → เลือก w_min = 5.05 USDT  (w_max = 50.00)

=== แผนวางกริด (อ่านจาก macro_montecarlo.csv) ===
ราคาปัจจุบัน (Spot): 3.112700 | ช่วงคาดการณ์ (Band): 2.247173 → 6.165107
จำนวนชั้นในแต่ละโซน (Zone counts): {'near': 1, 'mid': 1, 'far': 8, 'Total_levels': 10}
สรุปภาพรวม (Totals): {'total_usd': 100.01, 'est_net_pct_per_fill': 0.0092}

คำสั่งซื้อ (Orders):
ZONE | buy_price | usd_alloc | coin_size | tp_price | net_pct_est
 far | 2.247173 |    17.13 |  7.622018 | 2.269645 | 0.9200%
 far | 2.331139 |    15.95 |  6.840943 | 2.354450 | 0.9200%
 far | 2.464934 |    13.98 |  5.672799 | 2.489583 | 0.9200%
 far | 2.595495 |    11.94 |  4.602079 | 2.621450 | 0.9200%
 far | 2.701344 |    10.18 |  3.766886 | 2.728358 | 0.9200%
 far | 2.785283 |     8.67 |  3.113951 | 2.813136 | 0.9200%
 far | 2.876285 |     6.91 |  2.400772 | 2.905048 | 0.9200%
 far | 2.957